In [3]:
import os
import glob
import numpy as np
import pandas as pd
from autogluon.multimodal import MultiModalPredictor
from sklearn.preprocessing import LabelEncoder, StandardScaler

/home/salma.hassan/miniconda3/envs/auto_gpu/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [40]:
training = pd.read_csv('/home/salma.hassan/AI702/MaskAttention/features.csv')
testing = pd.read_csv('/home/salma.hassan/AI702/MaskAttention/features_test.csv')

In [41]:
label_encoder = LabelEncoder()

In [42]:
def description(cancer_type):
    return "A photo of leukemia cancer"

training['Description'] = training['Target'].apply(description)
testing['Description'] = testing['Target'].apply(description)

In [43]:
training['Target'] = label_encoder.fit_transform(training['Target'])
testing['Target'] = label_encoder.fit_transform(testing['Target'])

In [44]:
train_data = training
test_data = testing

In [45]:
from autogluon.tabular import FeatureMetadata
feature_metadata = FeatureMetadata.from_df(train_data)

print(feature_metadata)

('float', [])  : 96 | ['original_shape2D_Elongation', 'original_shape2D_MajorAxisLength', 'original_shape2D_MaximumDiameter', 'original_shape2D_MeshSurface', 'original_shape2D_MinorAxisLength', ...]
('int', [])    :  7 | ['original_shape2D_PixelSurface', 'original_firstorder_Energy', 'original_firstorder_Maximum', 'original_firstorder_Minimum', 'original_firstorder_Range', ...]
('object', []) :  2 | ['Image', 'Description']


In [46]:
label = 'Target'
image_col = 'Image'

In [47]:
feature_metadata = feature_metadata.add_special_types({"Description": ['text']})

print(feature_metadata)

('float', [])        : 96 | ['original_shape2D_Elongation', 'original_shape2D_MajorAxisLength', 'original_shape2D_MaximumDiameter', 'original_shape2D_MeshSurface', 'original_shape2D_MinorAxisLength', ...]
('int', [])          :  7 | ['original_shape2D_PixelSurface', 'original_firstorder_Energy', 'original_firstorder_Maximum', 'original_firstorder_Minimum', 'original_firstorder_Range', ...]
('object', [])       :  1 | ['Image']
('object', ['text']) :  1 | ['Description']


In [48]:
feature_metadata = feature_metadata.add_special_types({image_col: ['image_path']})

print(feature_metadata)

('float', [])              : 96 | ['original_shape2D_Elongation', 'original_shape2D_MajorAxisLength', 'original_shape2D_MaximumDiameter', 'original_shape2D_MeshSurface', 'original_shape2D_MinorAxisLength', ...]
('int', [])                :  7 | ['original_shape2D_PixelSurface', 'original_firstorder_Energy', 'original_firstorder_Maximum', 'original_firstorder_Minimum', 'original_firstorder_Range', ...]
('object', ['image_path']) :  1 | ['Image']
('object', ['text'])       :  1 | ['Description']


In [49]:
from autogluon.tabular.configs.hyperparameter_configs import get_hyperparameter_config
hyperparameters = get_hyperparameter_config('multimodal')

hyperparameters

{'NN_TORCH': {},
 'GBM': [{},
  {'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}},
  'GBMLarge'],
 'CAT': {},
 'XGB': {},
 'AG_AUTOMM': {},
 'VW': {}}

In [38]:
from autogluon.tabular import TabularPredictor
predictor = TabularPredictor(label=label).fit(
    train_data=train_data,
    hyperparameters=hyperparameters,
    feature_metadata=feature_metadata,
    time_limit=900,
)

No path specified. Models will be saved in: "AutogluonModels/ag-20240425_113053"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ... Time limit = 900s
AutoGluon will save models to "AutogluonModels/ag-20240425_113053"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.9.19
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Tue Sep 26 09:05:21 UTC 2023


In [41]:
leaderboard = predictor.leaderboard(test_data)

In [19]:
!pip install ray

In [42]:
leaderboard

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,NeuralNetTorch,0.609854,0.854,accuracy,0.053071,0.044838,8.033693,0.053071,0.044838,8.033693,1,True,5
1,LightGBMXT,0.603196,0.848,accuracy,0.030510,0.013040,14.111922,0.030510,0.013040,14.111922,1,True,2
2,WeightedEnsemble_L2,0.603196,0.862,accuracy,0.114396,0.064757,59.080343,0.005090,0.000812,0.091950,2,True,7
3,CatBoost,0.588549,0.844,accuracy,0.015235,0.010361,42.249317,0.015235,0.010361,42.249317,1,True,3
4,XGBoost,0.584554,0.848,accuracy,0.041000,0.008746,8.705383,0.041000,0.008746,8.705383,1,True,4
5,LightGBM,0.580559,0.850,accuracy,0.029383,0.012740,13.556908,0.029383,0.012740,13.556908,1,True,1
6,LightGBMLarge,0.580559,0.832,accuracy,0.098627,0.030213,36.682438,0.098627,0.030213,36.682438,1,True,6


In [52]:
from ray import tune

predictor_hpo = MultiModalPredictor(label="Target")
# "model.timm_image.checkpoint_name": tune.choice(["ghostnet_100", "mobilenetv3_large_100", "swin_base_patch4_window7_224", "convnext_base_in22ft1k"]),
# predictor.fit(hyperparameters={"model.names": ["hf_text", "timm_image", "clip", "categorical_mlp", "numerical_mlp", "fusion_mlp"]})

hyperparameters = {
            "optimization.learning_rate": tune.uniform(0.00005, 0.001),
            "optimization.optim_type": tune.choice(["adamw", "sgd"]),
            "optimization.max_epochs": tune.choice(["10", "20", "30"]), 
            "optimization.lr_schedule": tune.choice(["cosine_decay", "polynomial_decay", "linear_decay"]),
            "model.timm_image.checkpoint_name": tune.choice(["swin_base_patch4_window7_224", "convnext_base_in22ft1k"]),
            "optimization.top_k_average_method": "greedy_soup",
            "model.names": tune.choice([ "timm_image", "clip", "ft_transformer", "fusion_mlp"]),
            "model.ft_transformer.num_blocks": 7
}
hyperparameter_tune_kwargs = {
    "searcher": "bayes", # random
    "scheduler": "ASHA",
    "num_trials": 50,
}
predictor_hpo.fit(
        train_data=train_data,
        hyperparameters=hyperparameters,
        hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
    )


No path specified. Models will be saved in: "AutogluonModels/ag-20240425_144513"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.9.19
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Tue Sep 26 09:05:21 UTC 2023
CPU Count:          32
Pytorch Version:    2.1.2+cu121
CUDA Version:       12.1
Memory Avail:       101.96 GB / 125.55 GB (81.2%)
Disk Space Avail:   123535.23 GB / 238713.06 GB (51.8%)
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == int, but few unique label-values observed).
	5 unique label values:  [3, 4, 0, 2, 1]
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])


Trial name,should_checkpoint,val_accuracy
0041264e,True,0.662
01603ded,True,0.942
05192fd9,True,0.588
0a00c30c,True,0.882
0a78ed44,True,0.178
0fe7ea07,True,0.174
178a908d,True,0.848
19646bc5,True,0.182
2998c2a4,True,0.862
308a4dfa,True,0.182


Removing non-optimal trials and only keep the best one.
/home/salma.hassan/miniconda3/envs/auto_gpu/lib/python3.9/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name convnext_base_in22ft1k to current convnext_base.fb_in22k_ft_in1k.
  model = create_fn(
Start to fuse 3 checkpoints via the greedy soup algorithm.
/home/salma.hassan/miniconda3/envs/auto_gpu/lib/python3.9/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/salma.hassan/miniconda3/envs/auto_gpu/lib/pyth ...
/home/salma.hassan/miniconda3/envs/auto_gpu/lib/python3.9/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: sru

Predicting DataLoader 0: 100%|██████████| 16/16 [00:02<00:00,  7.20it/s]


/home/salma.hassan/miniconda3/envs/auto_gpu/lib/python3.9/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/salma.hassan/miniconda3/envs/auto_gpu/lib/pyth ...
/home/salma.hassan/miniconda3/envs/auto_gpu/lib/python3.9/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/salma.hassan/miniconda3/envs/auto_gpu/lib/pyth ...


Predicting DataLoader 0: 100%|██████████| 16/16 [00:02<00:00,  7.39it/s]


/home/salma.hassan/miniconda3/envs/auto_gpu/lib/python3.9/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/salma.hassan/miniconda3/envs/auto_gpu/lib/pyth ...
/home/salma.hassan/miniconda3/envs/auto_gpu/lib/python3.9/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/salma.hassan/miniconda3/envs/auto_gpu/lib/pyth ...


Predicting DataLoader 0: 100%|██████████| 16/16 [00:02<00:00,  7.41it/s]


AutoMM has created your model. 🎉🎉🎉

To load the model, use the code below:
    ```python
    from autogluon.multimodal import MultiModalPredictor
    predictor = MultiModalPredictor.load("/home/salma.hassan/AI702/MaskAttention/AutogluonModels/ag-20240425_144513")
    ```

If you are not satisfied with the model, try to increase the training time, 
adjust the hyperparameters (https://auto.gluon.ai/stable/tutorials/multimodal/advanced_topics/customization.html),
or post issues on GitHub (https://github.com/autogluon/autogluon/issues).




In [54]:
predictor_hpo = MultiModalPredictor.load("/home/salma.hassan/AI702/MaskAttention/AutogluonModels/ag-20240425_144513")
scores = predictor_hpo.evaluate(testing, metrics=["accuracy", "balanced_accuracy", "f1_weighted", "precision_weighted", "recall_weighted"])
scores

/home/salma.hassan/miniconda3/envs/auto_gpu/lib/python3.9/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name convnext_base_in22ft1k to current convnext_base.fb_in22k_ft_in1k.
  model = create_fn(
Load pretrained checkpoint: /home/salma.hassan/AI702/MaskAttention/AutogluonModels/ag-20240425_144513/model.ckpt
/home/salma.hassan/miniconda3/envs/auto_gpu/lib/python3.9/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/salma.hassan/miniconda3/envs/auto_gpu/lib/pyth ...
/home/salma.hassan/miniconda3/envs/auto_gpu/lib/python3.9/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun 

Predicting DataLoader 0: 100%|██████████| 24/24 [00:03<00:00,  7.25it/s]


{'accuracy': 0.715046604527297,
 'balanced_accuracy': 0.7054891637828741,
 'f1_weighted': 0.7300518829497755,
 'precision_weighted': 0.7711878872002697,
 'recall_weighted': 0.715046604527297}

In [53]:
predictor_hpo = MultiModalPredictor.load("/home/salma.hassan/AI702/MaskAttention/AutogluonModels/ag-20240425_133025")
scores = predictor_hpo.evaluate(testing, metrics=["accuracy", "balanced_accuracy", "f1_weighted", "precision_weighted", "recall_weighted"])
scores

/home/salma.hassan/miniconda3/envs/auto_gpu/lib/python3.9/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name convnext_base_in22ft1k to current convnext_base.fb_in22k_ft_in1k.
  model = create_fn(
Load pretrained checkpoint: /home/salma.hassan/AI702/MaskAttention/AutogluonModels/ag-20240425_133025/model.ckpt
/home/salma.hassan/miniconda3/envs/auto_gpu/lib/python3.9/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/salma.hassan/miniconda3/envs/auto_gpu/lib/pyth ...
/home/salma.hassan/miniconda3/envs/auto_gpu/lib/python3.9/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun 

Predicting DataLoader 0: 100%|██████████| 24/24 [00:09<00:00,  2.53it/s]


{'accuracy': 0.7336884154460719,
 'balanced_accuracy': 0.7489728351849223,
 'f1_weighted': 0.7569923924906933,
 'precision_weighted': 0.8175700298407225,
 'recall_weighted': 0.7336884154460719}

In [57]:
from autogluon.multimodal import MultiModalPredictor
label_col = "Target"
predictor = MultiModalPredictor(problem_type="multiclass", label=label_col)
predictor.fit(train_data, hyperparameters = {"model.timm_image.checkpoint_name": "convnext_base_in22ft1k", "optimization.max_epochs": 50 })

No path specified. Models will be saved in: "AutogluonModels/ag-20240425_201950"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.9.19
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Tue Sep 26 09:05:21 UTC 2023
CPU Count:          32
Pytorch Version:    2.1.2+cu121
CUDA Version:       12.1
Memory Avail:       71.36 GB / 125.55 GB (56.8%)
Disk Space Avail:   123181.84 GB / 238713.09 GB (51.6%)



AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir /home/salma.hassan/AI702/MaskAttention/AutogluonModels/ag-20240425_201950
    ```

[rank: 0] Seed set to 0
/home/salma.hassan/miniconda3/envs/auto_gpu/lib/python3.9/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name convnext_base_in22ft1k to current convnext_base.fb_in22k_ft_in1k.
  model = create_fn(
GPU Count: 1
GPU Count to be Used: 1
GPU 0 Name: Quadro RTX 6000
GPU 0 Memory: 0.8GB/24.0GB (Used/Total)

/home/salma.hassan/miniconda3/envs/auto_gpu/lib/python3.9/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/salma.hassan/miniconda3/envs/auto_gpu/lib/p

Epoch 0:  50%|████▉     | 138/277 [00:13<00:13, 10.36it/s]                 

Epoch 0, global step 8: 'val_accuracy' reached 0.36800 (best 0.36800), saving model to '/home/salma.hassan/AI702/MaskAttention/AutogluonModels/ag-20240425_201950/epoch=0-step=8.ckpt' as top 3


Epoch 0: 100%|█████████▉| 276/277 [00:45<00:00,  6.08it/s]

...

/home/salma.hassan/miniconda3/envs/auto_gpu/lib/python3.9/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/salma.hassan/miniconda3/envs/auto_gpu/lib/pyth ...
/home/salma.hassan/miniconda3/envs/auto_gpu/lib/python3.9/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/salma.hassan/miniconda3/envs/auto_gpu/lib/pyth ...


Predicting DataLoader 0: 100%|██████████| 16/16 [00:02<00:00,  7.56it/s]


/home/salma.hassan/miniconda3/envs/auto_gpu/lib/python3.9/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/salma.hassan/miniconda3/envs/auto_gpu/lib/pyth ...
/home/salma.hassan/miniconda3/envs/auto_gpu/lib/python3.9/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/salma.hassan/miniconda3/envs/auto_gpu/lib/pyth ...


Predicting DataLoader 0: 100%|██████████| 16/16 [00:02<00:00,  7.51it/s]


AutoMM has created your model. 🎉🎉🎉

To load the model, use the code below:
    ```python
    from autogluon.multimodal import MultiModalPredictor
    predictor = MultiModalPredictor.load("/home/salma.hassan/AI702/MaskAttention/AutogluonModels/ag-20240425_201950")
    ```

If you are not satisfied with the model, try to increase the training time, 
adjust the hyperparameters (https://auto.gluon.ai/stable/tutorials/multimodal/advanced_topics/customization.html),
or post issues on GitHub (https://github.com/autogluon/autogluon/issues).




In [58]:
scores = predictor.evaluate(testing, metrics=["accuracy"])
scores

/home/salma.hassan/miniconda3/envs/auto_gpu/lib/python3.9/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/salma.hassan/miniconda3/envs/auto_gpu/lib/pyth ...
/home/salma.hassan/miniconda3/envs/auto_gpu/lib/python3.9/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/salma.hassan/miniconda3/envs/auto_gpu/lib/pyth ...


Predicting DataLoader 0: 100%|██████████| 24/24 [00:06<00:00,  3.51it/s]


{'accuracy': 0.7243675099866844}